In [ ]:
# %%

from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.tabular.configs.hyperparameter_configs import get_hyperparameter_config

import numpy as np

from sklearnex import patch_sklearn
patch_sklearn()

label = 'signature'

data_url = 'https://raw.githubusercontent.com/mli/ag-docs/main/knot_theory/'

train_data = TabularDataset(f'{data_url}train.csv')
test_data = TabularDataset(f'{data_url}test.csv')

train_data.head()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


,Unnamed: 0,chern_simons,cusp_volume,hyperbolic_adjoint_torsion_degree,hyperbolic_torsion_degree,injectivity_radius,longitudinal_translation,meridinal_translation_imag,meridinal_translation_real,short_geodesic_imag_part,short_geodesic_real_part,Symmetry_0,Symmetry_D3,Symmetry_D4,Symmetry_D6,Symmetry_D8,Symmetry_Z/2 + Z/2,volume,signature
0,70746,0.090530,12.226322,0,10,0.507756,10.685555,1.144192,-0.519157,-2.760601,1.015512,0.0,0.0,0.0,0.0,0.0,1.0,11.393225,-2
1,240827,0.232453,13.800773,0,14,0.413645,10.453156,1.320249,-0.158522,-3.013258,0.827289,0.0,0.0,0.0,0.0,0.0,1.0,12.742782,0
2,155659,-0.144099,14.761030,0,14,0.436928,13.405199,1.101142,0.768894,2.233106,0.873856,0.0,0.0,0.0,0.0,0.0,0.0,15.236505,2
3,239963,-0.171668,13.738019,0,22,0.249481,27.819496,0.493827,-1.188718,-2.042771,0.498961,0.0,0.0,0.0,0.0,0.0,0.0,17.279890,-8
4,90504,0.235188,15.896359,0,10,0.389329,15.330971,1.036879,0.722828,-3.056138,0.778658,0.0,0.0,0.0,0.0,0.0,0.0,16.749298,4


In [ ]:
# %%

custom_hyperparameters = get_hyperparameter_config('light')
# custom_hyperparameters['GBM'] = [{'objective':'multiclass', 'boosting_type':'dart', 'n_estimators':5000}]
# custom_hyperparameters['CAT'] = [{'loss_function':'Logloss', 'iterations':5000}]
# custom_hyperparameters['RF'] = [{'criterion': 'log_loss', 'n_estimators':5000,'ag_args': {'name_suffix': 'LogLoss', 'problem_types': ['multiclass']}}]
# custom_hyperparameters['XT'] = [{'criterion': 'log_loss', 'n_estimators':5000,'ag_args': {'name_suffix': 'LogLoss', 'problem_types': ['multiclass']}}]
custom_hyperparameters['LR'] = [
    {'multi_class':'multinomial', 'penalty':None, 'tol':1e-6, 'max_iter':10000,
        'ag_args': {'name_suffix': 'Base'}},
    {'multi_class':'multinomial', 'penalty':'l2', 'tol':1e-6, 'max_iter':10000, 'C':0.5,
        'ag_args': {'name_suffix': 'Ridge'}},
    {'multi_class':'multinomial', 'penalty':'l1', 'tol':1e-6, 'max_iter':10000, 'C':0.5,
        'ag_args': {'name_suffix': 'Lasso'}},
    {'multi_class':'multinomial', 'penalty':'elasticnet', 'tol':1e-6, 'max_iter':10000, 'C':0.5,
        'ag_args': {'name_suffix': 'ElasticNet'}}
    ]

custom_preset = {'auto_stack': False, 'dynamic_stacking': False,
                'hyperparameters':custom_hyperparameters, 'refit_full': False,
                'set_best_to_refit_full': False, 'save_bag_folds': False,
                'num_cpus':6}

In [ ]:
# %%

np.random.seed(2024)

predictor = TabularPredictor(label=label, problem_type='multiclass', eval_metric='log_loss', log_to_file=True)

predictor.fit(train_data, presets=custom_preset, excluded_model_types=['KNN', 'NN_TORCH'])  

No path specified. Models will be saved in: "AutogluonModels\ag-20240817_222050"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22635
CPU Count:          8
Memory Avail:       7.53 GB / 15.70 GB (48.0%)
Disk Space Avail:   755.62 GB / 952.43 GB (79.3%)
Presets specified: [{'auto_stack': False, 'dynamic_stacking': False, 'hyperparameters': {'NN_TORCH': {}, 'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, 'GBMLarge'], 'CAT': {}, 'XGB': {}, 'FASTAI': {}, 'RF': [{'criterion': 'gini', 'max_depth': 15, 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'max_depth': 15, 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'max_depth': 15, 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regre

In [ ]:
# %%

metrics = ['model', 'score_test', 'score_val', 'eval_metric', 'pred_time_test', 'fit_time']

df_leaders = predictor.leaderboard(test_data)

df_leaders.head(40)[metrics]

,model,score_test,score_val,eval_metric,pred_time_test,fit_time
0,WeightedEnsemble_L2,-0.142122,-0.128937,log_loss,0.668103,216.731205
1,CatBoost,-0.158654,-0.150310,log_loss,0.147584,184.602990
2,NeuralNetFastAI,-0.168758,-0.162475,log_loss,0.109397,22.261913
3,XGBoost,-0.171801,-0.144754,log_loss,0.396110,9.132326
4,LightGBM,-0.184182,-0.161321,log_loss,0.218596,5.897210
5,LightGBMXT,-0.193761,-0.178918,log_loss,0.690213,8.712244
6,RandomForestEntr,-0.199264,-0.198239,log_loss,0.326667,3.541137
7,RandomForestGini,-0.214757,-0.213908,log_loss,0.485715,2.492254
8,LightGBMLarge,-0.316327,-0.290976,log_loss,0.262065,10.746322
9,ExtraTreesEntr,-0.374882,-0.378358,log_loss,0.375059,2.731819


In [ ]:
# %%

y_pred = predictor.predict_proba(test_data.drop(columns=[label]))
y_pred.head()

# predictor.evaluate(test_data, silent=True)

,-12,-10,-8,-6,-4,-2,0,2,4,6,8,10,12
0,0.000021,0.000009,1.613870e-05,3.275709e-04,9.994113e-01,0.000151,0.000027,0.000005,7.239689e-06,9.433326e-06,7.407968e-06,2.916414e-06,0.000004
1,0.000039,0.000022,1.693607e-05,1.572352e-05,3.720517e-04,0.616242,0.382921,0.000252,3.902075e-05,1.807626e-05,2.057650e-05,1.683622e-05,0.000025
2,0.000003,0.000001,6.674409e-07,6.662174e-07,6.185542e-07,0.000279,0.999707,0.000004,5.808662e-07,9.875779e-07,7.757728e-07,9.392224e-07,0.000001
3,0.000002,0.000001,1.395582e-06,4.646905e-05,2.394601e-05,0.000001,0.000003,0.000029,9.946107e-01,5.253551e-03,2.346622e-05,1.270021e-06,0.000003
4,0.000002,0.000002,1.201364e-06,7.925847e-07,7.766458e-07,0.000002,0.021628,0.978091,2.647080e-04,1.231660e-06,9.081442e-07,1.507803e-06,0.000003
